# Dense Optical Flow with Gunnar Farnebäck's Algorithm

Dense optical flow identifies the movement of each pixel across a series of images. In
this notebook, I'll build Farnebäck's algorithm from the ground up.

**RULES:** As usual, **`OpenCV`** is banned in this repository.

**References:**

-   [1]
    [Polynomial Expansion for Orientation and Motion Estimation](https://www.ida.liu.se/ext/WITAS-ev/Computer_Vision_Technologies/PaperInfo/farneback02.html)

-   [2]
    [Two-Frame Motion Estimation Based on Polynomial Expansion](https://link.springer.com/chapter/10.1007/3-540-45103-X_50)

-   [3]
    [Optical Flow - Michael Black - MLSS 2013 Tübingen](https://www.youtube.com/watch?v=tIwpDuqJqcE)

-   [4]
    [ericPrince's Pure python implementation of Gunnar Farneback's optical flow algorithm](https://github.com/ericPrince/optical-flow)

**Important Note:** Although [2] is the most widely cited paper on Farnebäck's
algorithm, a thorough comprehension of the algorithm requires reading Gunnar Farnebäck's
complete thesis [1].


In [ ]:
import numpy as np
from scipy.ndimage import correlate1d
from functools import partial
import skimage
import my_utils

# np.set_printoptions(threshold=np.inf)

## Motivation

In the world of computer vision, tracking the displacement of pixels across consecutive
frames is usually based on two assumptions:

1. **Brightness Constancy:** It states that the brightness of a pixel remains constant
   between consecutive images, despite its movement to a new position. This idea is
   captured by the equation:

    $$
    I(x + u, y + v, t + 1) = I(x, y, t)
    $$

    Here, $u$ and $v$ represent the horizontal and vertical shifts in the pixel's
    position, illustrating that the pixel's intensity doesn't change as it moves.

2. **Spatial Smoothness:** It is based on the idea that neighboring pixels usually move
   in a similar fashion because they are likely part of the same surface. Surfaces tend
   to be smooth, implying that adjacent pixels will have comparable motion. This concept
   is summarized as follows:

    $$
    u_p = u_n \quad \text{and} \quad v_p = v_n, \quad \forall n \in G(p)
    $$

    It means that the movement of a pixel $p$ in both horizontal ($u$) and vertical
    ($v$) directions is similar to that of its neighboring pixel $n$, suggesting that
    optical flow changes smoothly across the image.

**Objective Function:**

Derived from these assumptions, we can define the objective functions as follow:

-   **Brightness Constancy:**

    $$
    E_D(u, v) = \sum (I(x + u, y + v, t + 1) - I(x, y, t))^2
    $$

    This equation emphasizes that deviations from brightness constancy are minimized,
    assuming the presence of Gaussian noise.

-   **Spatial Smoothness:**

    $$
    E_s(u, v) = \sum(u_p - u_n)^2 + \sum(v_p - v_n)^2, \quad \forall n \in G(p)
    $$

    This formula encourages smoothness by penalizing variations in the motion between a
    pixel and its neighbors.

**Solving the Equations**

The principle of brightness constancy implies that a pixel's intensity does not change
over time as it moves. This is mathematically represented as:

$$
I(x, y, t) = I(x + \Delta x, y + \Delta y, t + \Delta t)
$$

For slight movements ($\Delta x$, $\Delta y$, $\Delta t$), we can use the first-order
Taylor series expansion for image intensity $I$:

$$
I(x + \Delta x, y + \Delta y, t + \Delta t) \approx I(x, y, t) + \frac{\partial I}{\partial x} \Delta x + \frac{\partial I}{\partial y} \Delta y + \frac{\partial I}{\partial t} \Delta t
$$

By applying the brightness constancy condition and simplifying, we eliminate the term
$I(x, y, t)$ on both sides:

$$
I_x \Delta x + I_y \Delta y + I_t \Delta t = 0
$$

Dividing through by $\Delta t$ (assuming it is not zero) and using
$\Delta x / \Delta t = u$ and $\Delta y / \Delta t = v$ for velocity components, we
arrive at the optical flow constraint equation:

$$
I_x u + I_y v + I_t = 0
$$


## Polynomial Expansion for Orientation and Motion Estimation

Polynomial expansion fits polynomials to local pixel neighborhoods to approximate image
motion and structure as detailed in [1]. It uses quadratic, linear, and constant terms
(A, B, C) to describe local image structure and motion:

-   **Quadratic Term (A):** Represents the curvature of the image intensity surface,
    indicating the geometric structure like edges or flat areas. High values signal
    significant curvature or rapid intensity changes.

-   **Linear Term (B):** Represents the gradient at each pixel, showing the direction
    and magnitude of the most substantial intensity change, crucial for identifying
    motion direction and feature orientation.

-   **Constant Term (C):** Represents the average intensity in a pixel's neighborhood,
    indicating the area's overall brightness or darkness without detailing structure or
    motion.


### Signal, Certainty, and Applicability

The signal $f$ and its local approximation $\hat{f}$, a finite-dimensional vector in
$C^n$, reflect the signal within a specific neighborhood, represented as an $n \times 1$
column vector.

**Certainty** is the confidence in signal values, indicated by non-negative real
numbers. The field of certainty is $c$, with $\hat{c}$ as the $n \times 1$ vector for
local certainty levels.

**Applicability** defines the relevance of basis functions within the neighborhood,
expressed as non-negative values in an $n \times 1$ vector, $a$. Unlike certainty,
applicability focuses on the significance of each point, with non-zero values indicating
relevance. While intuitively applicability might range between $[0, 1]$, no such
restriction is necessary, as the scale of these values doesn't impact their relevance.

For generating applicability and certainty specifics, see Section 3.10, Section 4.2 and
Section 4.3 of [1]. The functions `generate_applicability` and `generate_certainty`
detail their implementation.


In [ ]:
def generate_applicability(sigma):
    """Calculate 1D Gaussian applicability kernel."""
    n = int(4 * sigma + 1)  # Capture significant parts of the Gaussian distribution
    x = np.arange(-n, n + 1)
    applicability = np.exp(-(x**2) / (2 * sigma**2))
    return x, applicability


def generate_certainty(height, width, denominator=5):
    """should it be gaussion or linear"""
    kernel = np.minimum(
        1,
        1 / denominator * np.minimum(np.arange(height)[:, None], np.arange(width)),
    )
    kernel = np.minimum(
        kernel,
        1
        / denominator
        * np.minimum(
            height - 1 - np.arange(height)[:, None],
            width - 1 - np.arange(width),
        ),
    )
    return kernel

### Separable Correlation

Separable correlation is a technique that enhances computational efficiency in signal
processing and image analysis by simplifying correlation operations, especially with
large filters. It involves decomposing a two-dimensional kernel into two orthogonal
one-dimensional kernels, allowing two-dimensional correlation to be performed as two
separate one-dimensional processes: first across rows, then down columns, or vice versa.

Mathematically, if a two-dimensional kernel $H$ can be expressed as the outer product of
two one-dimensional vectors $u$ and $v$:

$$
H = u \otimes v
$$

then the kernel is separable. For an input signal or image $I$, the two-dimensional
correlation with a separable kernel $H$ occurs in two phases:

1. **Horizontal Pass:** Apply one-dimensional kernel $u$ across each row of $I$.
2. **Vertical Pass:** Apply one-dimensional kernel $v$ down each column of the
   horizontal pass result.


### Defining `poly_exp`

To construct the polynomial expansion function `poly_exp`, we follow the guidelines of
Chapter 4 in [1]. The process is divided into four main parts:

-   **Initialization and Applicability Generation:** Detailed on Section 4.3, this step
    involves setting up the initial conditions and generating the applicability matrix
    based on $\sigma$.

-   **Calculation of Polynomial Coefficients `b`:** As described in Section 4.3, this
    involves calculating the polynomial expansion coefficients for horizontal
    (x-direction) and vertical (y-direction) correlations separately. This step also
    includes multiplying the certainty map with the image signal to prioritize signal
    values based on their certainty.

-   **Cross-Correlation and Polynomial Parameters Calculation:** Utilizes matrices `G`
    and `v` to solve for `r` using Equation (4.9), where $r = G^{-1}v$. Here, `r`
    represents the parameters of the second-order polynomial modeling the signal `f`.

-   **Final Polynomial Terms Extraction:** Based on Equation (4.3), the quadratic (`A`),
    linear (`B`), and constant (`C`) terms of the polynomial are derived, respectively.

Finally, the dimensions of the coefficients and residual error calculation are
summarized as follows:

-   Coefficient `b`: (n, n, 6)
-   Parameters `r`: (f, f, 6)
-   Signal `f`: (f, f)
-   Residual error $e$ is calculated as $\|b \cdot r - f\|_W$, as shown in equation
    3.19.


In [ ]:
def poly_exp(f, certainty, sigma):
    # Initialization and Applicability Generation
    height, width = f.shape
    x, applicability = generate_applicability(sigma)

    # Calculation of Polynomial Coefficients `b`
    bx = np.stack(
        [
            np.ones(applicability.shape),
            x,
            np.ones(applicability.shape),
            x**2,
            np.ones(applicability.shape),
            x,
        ],
        axis=-1,
    )  # (n, 6)
    by = np.stack(
        [
            np.ones(applicability.shape),
            np.ones(applicability.shape),
            x,
            np.ones(applicability.shape),
            x**2,
            x,
        ],
        axis=-1,
    )  # (n, 6)

    # Pre-calculate product of certainty and signal
    cf = certainty * f

    # Cross-Correlation and Polynomial Parameters Calculation
    G = np.empty(list(f.shape) + [bx.shape[-1]] * 2)  # (height, width, 6, 6)
    v = np.empty(list(f.shape) + [bx.shape[-1]])  # (height, width, 6)

    # Apply separable cross-correlations
    # Pre-calculate quantities.
    ab = np.einsum("i,ij->ij", applicability, bx)
    abb = np.einsum("ij,ik->ijk", ab, bx)
    # Calculate G and v for each pixel with cross-correlation
    for i in range(bx.shape[-1]):
        for j in range(bx.shape[-1]):
            G[..., i, j] = correlate1d(
                certainty, abb[..., i, j], axis=0, mode="constant", cval=0
            )

        v[..., i] = correlate1d(cf, ab[..., i], axis=0, mode="constant", cval=0)

    # Pre-calculate quantities.
    ab = np.einsum("i,ij->ij", applicability, by)
    abb = np.einsum("ij,ik->ijk", ab, by)
    # Calculate G and v for each pixel with cross-correlation
    for i in range(bx.shape[-1]):
        for j in range(bx.shape[-1]):
            G[..., i, j] = correlate1d(
                G[..., i, j], abb[..., i, j], axis=1, mode="constant", cval=0
            )

        v[..., i] = correlate1d(v[..., i], ab[..., i], axis=1, mode="constant", cval=0)

    # Solve r for each pixel
    r = np.linalg.solve(G, v)

    # Final Polynomial Terms Extraction
    # Quadratic term
    A = np.empty(list(f.shape) + [2, 2])
    A[..., 0, 0] = r[..., 3]
    A[..., 0, 1] = r[..., 5] / 2
    A[..., 1, 0] = A[..., 0, 1]
    A[..., 1, 1] = r[..., 4]

    # Linear term
    B = np.empty(list(f.shape) + [2])
    B[..., 0] = r[..., 1]
    B[..., 1] = r[..., 2]

    # constant term
    C = r[..., 0]

    return A, B, C

After defining the polynomial expansion function `poly_exp`, we can visualize the terms
`A`, `B`, and `C` as follows. The quadratic term `A` captures the even part of the
signal, while the linear term `B` encapsulates the odd part. Meanwhile, `C` reflects
variations in the local differential convolution level.


In [ ]:
import cv2

img = cv2.imread("input/yosemite/yos02.tif")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print("img:", img.shape, img.dtype, img.min(), img.max())
# TODO: Use float64 should also work. Input image should be floating point.
# gray = gray.astype(np.float64)
# gray /= 255
print(f"gray: {gray.shape}, {gray.dtype}, {gray.min()}, {gray.max()}")

height, width, *_ = img.shape
certainty = generate_certainty(height, width, denominator=5)
print(f"certainty: {certainty.shape}, {certainty.dtype}, {certainty.min()}, {certainty.max()}")

A, B, C = poly_exp(f=gray, certainty=certainty, sigma=4)
print("A:", A.shape)
print("B:", B.shape)
print("C:", C.shape)

my_utils.visualize_polynomial_expansion(
    img, A, B, C, out_path="./output/polynomial_expansion.png"
)

## Displacement Estimation

Polynomial expansion allows for approximating the neighborhood of a pixel. When
considering an ideal translation, we analyze the behavior of a quadratic polynomial
signal:

$$
f_1(\mathbf{x}) = \mathbf{x}^T \mathbf{A}_1 \mathbf{x} + \mathbf{b}_1^T \mathbf{x} + c_1
$$

and construct a new signal $f_2$ displaced globally by $\mathbf{d}$, as shown in
Equation (7.2) in [1]:

$$
f_2(\mathbf{x}) = f_1(\mathbf{x} - \mathbf{d}) = \mathbf{x}^T \mathbf{A}_2 \mathbf{x} + \mathbf{b}_2^T \mathbf{x} + c_2.
$$

By equating coefficients in the quadratic polynomials, we find:

-   $\mathbf{A}_2 = \mathbf{A}_1$,
-   $\mathbf{b}_2 = \mathbf{b}_1 - 2\mathbf{A}_1\mathbf{d}$,
-   $c_2 = \mathbf{d}^T \mathbf{A}_1 \mathbf{d} - \mathbf{b}_1^T \mathbf{d} + c_1$.

For non-singular $\mathbf{A}_1$, the translation $\mathbf{d}$ can be calculated as:

$$
\mathbf{d} = -\frac{1}{2} \mathbf{A}_1^{-1} (\mathbf{b}_2 - \mathbf{b}_1).
$$

This principle, as [1] states, is applicable regardless of signal dimensionality.

Furthermore, [1] suggests utilizing local polynomial approximations for each pixel
across images, implying the use of $\mathbf{A}_1(\mathbf{x})$,
$\mathbf{b}_1(\mathbf{x})$, and $c_1(\mathbf{x})$ for the first image, and
$\mathbf{A}_2(\mathbf{x})$, $\mathbf{b}_2(\mathbf{x})$, and $c_2(\mathbf{x})$ for the
second. Ideally, this would result in $\mathbf{A}_1 = \mathbf{A}_2$, but practically, an
approximation is used:

$$
\mathbf{A}(\mathbf{x}) = \frac{\mathbf{A}_1(\mathbf{x}) + \mathbf{A}_2(\mathbf{x})}{2}.
$$

Additionally, [1] defines

$$
\Delta \mathbf{b}(\mathbf{x}) = -\frac{1}{2}(\mathbf{b}_2(\mathbf{x}) - \mathbf{b}_1(\mathbf{x})),
$$

leading to the primary constraint for displacement estimation:

$$
\mathbf{A}(\mathbf{x})\mathbf{d}(\mathbf{x}) = \Delta \mathbf{b}(\mathbf{x}),
$$

where $\mathbf{d}(\mathbf{x})$ represents a spatially varying displacement field, moving
from a global to a local displacement context.


### Calculating Displacement over a Neighborhood

Section 7.3 in [1] indicates that solving the equation

$$
\mathbf{A}(\mathbf{x})\mathbf{d}(\mathbf{x}) = \Delta \mathbf{b}(\mathbf{x}),
$$

pointwise does not yield satisfactory results. To improve accuracy, [1] recommends
optimizing the solution across a neighborhood $I$ around $\mathbf{x}$. This is achieved
by minimizing the weighted sum:

$$
\sum_{\Delta \mathbf{x} \in I} w(\Delta \mathbf{x})\|\mathbf{A}(\mathbf{x} + \Delta \mathbf{x})\mathbf{d}(\mathbf{x}) - \Delta \mathbf{b}(\mathbf{x} + \Delta \mathbf{x})\|^2,
$$

where $w(\Delta \mathbf{x})$ serves as a weight function, enhancing the contribution of
more relevant points within the neighborhood. The optimal displacement
$\mathbf{d}(\mathbf{x})$ is found with:

$$
\mathbf{d}(\mathbf{x}) = \left(\sum w\mathbf{A}^T \mathbf{A}\right)^{-1} \sum w\mathbf{A}^T \Delta \mathbf{b},
$$

after simplifying the notation for clarity. The corresponding minimum error
$e(\mathbf{x})$, indicating the solution's accuracy, is expressed as:

$$
e(\mathbf{x}) = (\sum w\Delta \mathbf{b}^T \Delta \mathbf{b}) - \mathbf{d}(\mathbf{x})^T \left(\sum w\mathbf{A}^T \Delta \mathbf{b}\right).
$$

This formulation computes $\mathbf{A}^T \mathbf{A}$, $\mathbf{A}^T \Delta \mathbf{b}$,
and $\Delta \mathbf{b}^T \Delta \mathbf{b}$ pointwise, then averages these with the
weight $w$ before calculating the displacement. The error $e(\mathbf{x})$ inversely
relates to confidence: smaller values indicate greater reliability. The solution is both
existent and unique unless the neighborhood is uniformly affected by the aperture
problem.


### Estimating a Parameterized Displacement Field

Section 6.3.1 in [1] shows that the simplest possible motion model:

$$
v_x(x, y) = a,
$$

$$
v_y(x, y) = b,
$$

where $x$ and $y$ are the spatial coordinates, $v_x$ and $v_y$ are the $x$ and $y$
components of the velocity, and $a$ and $b$ are the model parameters. Geometrically,
this motion model assumes that the velocity is constant over the region and corresponds
to objects undergoing a pure translation under orthographic projection.

Equation (6.5) in [1] shows a more powerful alternative, the affine motion model:

$$
v_x(x, y) = ax + by + c,
$$

$$
v_y(x, y) = dx + ey + f,
$$

which applies to planar patches undergoing rigid body motion, i.e., translation plus
rotation, under orthographic projection. To also account for perspective projection, [1]
introduces the eight-parameter motion model in Equation (6.6):

$$
v_x(x, y) = a_1 + a_2 x + a_3 y + a_7 x^2 + a_8 xy,
$$

$$
v_y(x, y) = a_4 + a_5 x + a_6 y + a_7 xy + a_8 y^2.
$$

The usefulness of these models depends on the application, but it is useful to note
that, sufficiently far away, most surfaces can be approximated as planar. Moreover, if
the distance to the scene is much larger than the variation in distance within the
scene, perspective projection can be approximated by orthographic projection.

Following the motion model introduced above, we can enhance the robustness of
displacement estimation by parameterizing the displacement field according to the
eight-parameter motion model.

Section 7.5 in [1] rewrites the above formula to:

$$
d = Sp,
$$

where $S$ is the matrix containing the spatial variables,

$$
S = \left[ \begin{array}{cc}
1 & x & y & 0 & 0 & 0 & x^2 & xy \\
0 & 0 & 0 & 1 & x & y & xy & y^2 \\
\end{array} \right],
$$

and $p$ is the parameter vector,

$$
p = \left[ \begin{array}{c}
a_1 \\
a_2 \\
a_3 \\
a_4 \\
a_5 \\
a_6 \\
a_7 \\
a_8 \\
\end{array} \right].
$$

For the weighted least squares problem, we solve

$$
\sum w_i \|A_i S_i p - \Delta b_i\|^2,
$$

where $i$ indexes coordinates in a neighborhood. The solution is

$$
p = \left( \sum w_i S_i^T A_i^T A_i S_i \right)^{-1} \sum w_i S_i^T A_i^T \Delta b_i,
$$


In [ ]:
def motion_model(x, model):
    # Evaluate warp parametrization model at pixel coordinates
    if model == "constant":
        S = np.eye(2)

    elif model in ("affine", "eight_param"):
        # TODO: It seems affine and eight_param cannot produce valid results.
        # (height, width, 6 or 8)
        S = np.empty(list(x.shape) + [6 if model == "affine" else 8])

        S[..., 0, 0] = 1
        S[..., 0, 1] = x[..., 0]
        S[..., 0, 2] = x[..., 1]
        S[..., 0, 3] = 0
        S[..., 0, 4] = 0
        S[..., 0, 5] = 0

        S[..., 1, 0] = 0
        S[..., 1, 1] = 0
        S[..., 1, 2] = 0
        S[..., 1, 3] = 1
        S[..., 1, 4] = x[..., 0]
        S[..., 1, 5] = x[..., 1]

        if model == "eight_param":
            S[..., 0, 6] = x[..., 0] ** 2
            S[..., 0, 7] = x[..., 0] * x[..., 1]

            S[..., 1, 6] = x[..., 0] * x[..., 1]
            S[..., 1, 7] = x[..., 1] ** 2

    else:
        raise ValueError("Invalid parametrization model")

    return S

### Refining the Model with Prior Knowledge

One of the key challenges in estimating dense optical flow using polynomial expansions,
as proposed by Gunnar Farnebäck, is the underlying assumption that the local polynomial
approximations remain constant across two consecutive frames, differing only by a
displacement vector. However, since these polynomial models are local approximations,
their parameters can vary significantly across the image, leading to inaccuracies in the
constraint equation:

$$
A(\mathbf{x})\mathbf{d}(\mathbf{x}) = \Delta b(\mathbf{x}).
$$

This approximation holds relatively well for small displacements but becomes less
reliable for larger movements. To mitigate this, the algorithm does not restrict the
comparison of polynomials to identical coordinates in consecutive frames. With some
prior knowledge of the motion, it's possible to compare the polynomial at a point
$\mathbf{x}$ in the first frame to that at a point
$\mathbf{x} + \tilde{\mathbf{d}}(\mathbf{x})$ in the second frame, where
$\tilde{\mathbf{d}}(\mathbf{x})$ represents an initial, possibly rough, estimate of the
displacement, quantized to integer pixel values. This technique aims to refine the
estimate of the displacement vector by considering the difference between the real
displacement and this initial estimate, which should be smaller and more manageable.

This refinement is mathematically represented by modifying the equations to:

$$
A(\mathbf{x}) = \frac{A_1(\mathbf{x}) + A_2(\mathbf{x} + \tilde{\mathbf{d}}(\mathbf{x}))}{2},
$$

$$
\Delta b(\mathbf{x}) = -\frac{1}{2}(b_2(\mathbf{x} + \tilde{\mathbf{d}}(\mathbf{x})) - b_1(\mathbf{x})) + A(\mathbf{x}) \tilde{\mathbf{d}}(\mathbf{x}),
$$

where $\tilde{\mathbf{d}}(\mathbf{x})$ is the pre-estimated displacement vector. The
term $A(\mathbf{x}) \tilde{\mathbf{d}}(\mathbf{x})$ reintegrates the quantized prior
knowledge of displacement into the model. In the absence of any initial displacement
($\tilde{\mathbf{d}}(\mathbf{x}) = 0$), these equations simplify to their original
forms, as expected.

The algorithm, summarized in the last sections, is visualized in the following block
diagram. The inputs are the coefficients of the quadratic polynomial expansions ($A_1$,
$b_1$, $A_2$, $b_2$) of two consecutive frames and an initial displacement field
$\mathbf{d}_{in}$. The output is the refined displacement field $\mathbf{d}_{out}$.

<figure>
  <div style="display: flex; justify-content: space-between; max-width: 300px; /* Adjust this value as needed */">
    <div style="text-align: center;">
      <img src="./images/fig_7_8.png" style="max-width: 100%; height: auto;">
      <p><strong></strong></p>
      <p>Block diagram of the basic displacement estimation algorithm (DE) adapted from [1].</p>
    </div>
  </div>
</figure>


### Noise Reduction in Displacement Fields

A detailed analysis of the residual displacement field reveals that the predominant
sources of noise are regions lacking distinct structural features or with low contrast.
Section 7.10.3 in [1] mitigates this problem by "forcing" the background displacement
field onto these uncertain estimates. This is accomplished by introducing a
regularization term, i.e., minimizing the expression:

$$
\mu\|\mathbf{d}(\mathbf{x}) - \mathbf{d}_0 (\mathbf{x})\|^2 + \sum_{\Delta x \in I} w(\Delta x)\|\mathbf{A}(\mathbf{x} + \Delta \mathbf{x})\mathbf{d}(\mathbf{x}) - \Delta \mathbf{b}(\mathbf{x} + \Delta \mathbf{x})\|^2,
$$

where $\mathbf{d}_0$ represents the previously estimated background displacement field,
and $\mu$ is a weighting constant.

The solution to the equation above is given by Equation (7.36) in [1] as:

$$
\mathbf{d}(\mathbf{x}) = \left(\mu \mathbf{I} + \sum w\mathbf{A}^T \mathbf{A}\right)^{-1} \left(\mu \mathbf{d}_0(\mathbf{x}) + \sum w\mathbf{A}^T \Delta \mathbf{b}\right),
$$

The corresponding block diagram, illustrating the adaptation of the basic displacement
estimation algorithm to include this noise reduction technique, is designated below.

[1] suggests determining $\mu$ by taking the average of half the trace of
$\mathbf{G}_{avg}$ across the entire image, in accordance with the notation described in
the subsequent figure.

<figure>
  <div style="display: flex; justify-content: space-between; max-width: 300px; /* Adjust this value as needed */">
    <div style="text-align: center;">
      <img src="./images/fig_7_19.png" style="max-width: 100%; height: auto;">
      <p><strong></strong></p>
      <p>Block diagram of the revised basic displacement estimation algorithm (DE) incorporating regularization adapted from [1].</p>
    </div>
  </div>
</figure>


In [ ]:
def estimate_displacement_with_regularization(A, S_T, S, delB, w, mu):
    # Pre-calculate quantities recommended by paper
    A_T = A.swapaxes(-1, -2)
    ATA = S_T @ A_T @ A @ S
    ATb = (S_T @ A_T @ delB[..., None])[..., 0]
    # btb = delB.swapaxes(-1, -2) @ delB
    G_avg = np.mean(ATA, axis=(0, 1))
    h_avg = np.mean(ATb, axis=(0, 1))
    p_avg = np.linalg.solve(G_avg, h_avg)  # fig. 7.8
    d_avg = (S @ p_avg[..., None])[..., 0]

    # Default value for mu is to set mu to 1/2 the trace of G_avg
    if mu is None:
        mu = 1 / 2 * np.trace(G_avg)

    # Apply separable cross-correlation to calculate linear equation
    G = correlate1d(A_T @ A, w, axis=0, mode="constant", cval=0)
    # G = correlate1d(ATA, w, axis=0, mode="constant", cval=0)
    G = correlate1d(G, w, axis=1, mode="constant", cval=0)

    h = correlate1d((A_T @ delB[..., None])[..., 0], w, axis=0, mode="constant", cval=0)
    # h = correlate1d(ATb, w, axis=0, mode="constant", cval=0)
    h = correlate1d(h, w, axis=1, mode="constant", cval=0)

    # Refine estimate of displacement field
    # TODO: A bug when motion model is not constant
    d = np.linalg.solve(G + mu * np.eye(2), h + mu * d_avg)
    return d


def estimate_displacement_without_regularization(A, S_T, S, delB, w):
    # Pre-calculate quantities recommended by paper
    A_T = A.swapaxes(-1, -2)
    ATA = S_T @ A_T @ A @ S
    ATb = (S_T @ A_T @ delB[..., None])[..., 0]
    # btb = delB.swapaxes(-1, -2) @ delB

    # If mu is 0, it means the global/average parametrized warp should not be
    # calculated, and the parametrization should apply to the local calculations
    # if mu == 0: # page 132
    # Apply separable cross-correlation to calculate linear equation
    # for each pixel: G*d = h
    G = correlate1d(ATA, w, axis=0, mode="constant", cval=0)
    G = correlate1d(G, w, axis=1, mode="constant", cval=0)

    h = correlate1d(ATb, w, axis=0, mode="constant", cval=0)
    h = correlate1d(h, w, axis=1, mode="constant", cval=0)

    d = (S @ np.linalg.solve(G, h)[..., None])[..., 0]
    return d

### Iterative Displacement Estimation

Section 7.7.1 of [1] shows the simplest solution involves iterating the displacement
estimation process three times, as illustrated in the figure below.

The output displacement from one iteration serves as the a priori displacement for the
subsequent iteration. Initially, the a priori displacement field $\mathbf{d}_{in}$ is
typically set to zero, unless there is actual knowledge available about the displacement
field. The same polynomial expansion coefficients are used across all iterations and are
required to be computed only once.

The vulnerability of this method lies primarily in the first iteration. If the
displacements (relative to the a priori displacements) are excessively large, it is
unreasonable to anticipate improvements in the output displacements, rendering further
iterations **ineffective**.

<figure>
  <div style="display: flex; justify-content: space-between; max-width: 300px; /* Adjust this value as needed */">
    <div style="text-align: center;">
      <img src="./images/fig_7_9.png" style="max-width: 100%; height: auto;">
      <p><strong></strong></p>
      <p>Block diagram of the iterative displacement estimation algorithm adapted from [1].</p>
    </div>
  </div>
</figure>


In [ ]:
def flow_iterative(
    f1, f2, sigma, sigma_flow, num_iter=1, d=None, model="constant", mu=None
):

    # TODO: add initial warp parameters as optional input?

    height, width, *_ = f1.shape
    c1 = generate_certainty(height, width, 5)  # (height, width) float64 0.0 1.0
    c2 = generate_certainty(height, width, 5)  # (height, width) float64 0.0 1.0

    # Calculate the polynomial expansion at each point in the images
    A1, B1, C1 = poly_exp(f1, c1, sigma)
    A2, B2, C2 = poly_exp(f2, c2, sigma)

    # Pixel coordinates of each point in the images, (height, width, 2)
    x = np.stack(
        np.broadcast_arrays(np.arange(f1.shape[0])[:, None], np.arange(f1.shape[1])),
        axis=-1,
    ).astype(int)

    # Initialize displacement field
    if d is None:
        d = np.zeros(list(f1.shape) + [2])  # (height, width, 2)

    # Set up applicability convolution window
    n_flow = int(4 * sigma_flow + 1)
    xw = np.arange(-n_flow, n_flow + 1)
    w = np.exp(-(xw**2) / (2 * sigma_flow**2))

    S = motion_model(x, model)

    S_T = S.swapaxes(-1, -2)

    # Iterate convolutions to estimate the optical flow
    for _ in range(num_iter):
        # Set d~ as displacement field fit to nearest pixel (and constrain to not
        # being off image). Note we are setting certainty to 0 for points that
        # would have been off-image had we not constrained them
        d_ = d.astype(int)  # priori displacement
        x_ = x + d_

        # x_ = np.maximum(np.minimum(x_, np.array(f1.shape) - 1), 0)

        # Constrain d~ to be on-image, and find points that would have
        # been off-image
        x_2 = np.maximum(np.minimum(x_, np.array(f1.shape) - 1), 0)
        off_f = np.any(x_ != x_2, axis=-1)
        x_ = x_2

        # Set certainty to 0 for off-image points
        c_ = c1[x_[..., 0], x_[..., 1]]
        c_[off_f] = 0

        # Calculate A and delB for each point, according to paper
        A = (A1 + A2[x_[..., 0], x_[..., 1]]) / 2
        A *= c_[
            ..., None, None
        ]  # recommendation in paper: add in certainty by applying to A and delB

        delB = -1 / 2 * (B2[x_[..., 0], x_[..., 1]] - B1) + (A @ d_[..., None])[..., 0]
        delB *= c_[
            ..., None
        ]  # recommendation in paper: add in certainty by applying to A and delB

        if mu == 0:
            d = estimate_displacement_without_regularization(A, S_T, S, delB, w)
        else:
            d = estimate_displacement_with_regularization(A, S_T, S, delB, w, mu)

    return d

### Multi-scale Displacement Estimation

Section 7.7.2 in [1] introduces a multi-scale strategy for managing large displacements
through coarse to fine scales, albeit with a trade-off in precision initially. This
approach is illustrated in the subsequent figure.

The process begins at the coarsest scale, where a preliminary, albeit rough,
displacement estimate, $\mathbf{d}_{\text{in}}$, is commonly assumed to be zero in the
absence of pre-existing displacement information. This estimate is incrementally refined
across finer scales to achieve more accurate displacement detection. Unlike single-scale
iterative methods, the multi-scale approach needs to recalculate polynomial expansion
coefficients at each scale.

The function `gen_gaussian_pyramids`, detailed below, create a Gaussian pyramid. Each
image in this pyramid is represented as a floating-point number within the 0 to 1 range,
ensuring uniformity in subsequent processing steps.

<figure>
  <div style="display: flex; justify-content: space-between; max-width: 300px; /* Adjust this value as needed */">
    <div style="text-align: center;">
      <img src="./images/fig_7_10.png" style="max-width: 100%; height: auto;">
      <p><strong></strong></p>
      <p>Block diagram of the multi-scale displacement estimation algorithm adapted from [1].</p>
    </div>
  </div>
</figure>


In [ ]:
def gen_gaussian_pyramids(img_list, n_pyr):
    """
    Applies Gaussian pyramid transformations to a list of images, zips the transformed images together,
    and then reverses the order of the resulting list.

    Parameters:
    - img_list: List of images to transform. Each image should be compatible with skimage.transform.pyramid_gaussian.
    - n_pyr: The number of pyramid layers to use in the transformation.

    Returns:
    - A reversed list of the zipped, pyramid-transformed images.
    """
    # Apply the Gaussian pyramid transformation to each image in the list with the specified number of layers
    transformed_images = list(
        map(partial(skimage.transform.pyramid_gaussian, max_layer=n_pyr), img_list)
    )

    # Zip the transformed images together and reverse the order
    zipped_and_reversed = reversed(list(zip(*transformed_images)))

    return zipped_and_reversed

## Putting It All Together

After defining all the components of Farnebäck's algorithm, we can encapsulate the
entire process into a single function. The `calc_optical_flow_farneback` function takes
two images as inputs and computes the optical flow between them.


In [ ]:
def calc_optical_flow_farneback(img1, img2):
    assert img1.shape == img2.shape

    n_pyr = 3
    opts = dict(
        sigma=4.0,
        sigma_flow=4.0,
        num_iter=3,
        model="constant",  # TODO: Only constant works.
        mu=None,  # if mu is zero, there will be singularity error in linalg error. Use None and let it be computed automatically
    )

    d = None  # optical flow field

    gaussion_pyramids = gen_gaussian_pyramids([img1, img2], n_pyr=n_pyr)
    for pyr1, pyr2 in gaussion_pyramids:
        if d is not None:
            d = skimage.transform.pyramid_expand(d, channel_axis=-1)
            d = d[: pyr1.shape[0], : pyr2.shape[1]] * 2
            # TODO: d and pyr1 shape may not match due to downsampling/upsampling.
            assert d.shape[0] == pyr1.shape[0] and d.shape[1] == pyr1.shape[1]

        d = flow_iterative(pyr1, pyr2, d=d, **opts)

    return d

In [ ]:
import cv2

DEBUG = True

cap = cv2.VideoCapture(cv2.samples.findFile("input/snatch.mp4"))  # 25 fps
ret, frame1 = cap.read()
prev = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
# prev: (1080, 1920) uint8 0 255
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec
print("cap.get(cv2.CAP_PROP_FPS)", cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter(
    "./output/snatch_optical_flow.mp4",
    fourcc,
    fps=cap.get(cv2.CAP_PROP_FPS),
    frameSize=(frame1.shape[1], frame1.shape[0]),
)

stop = 1
while 1:
    ret, frame2 = cap.read()
    if not ret or stop == 0:
        print("No frames grabbed!")
        break
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    # next: (1080, 1920) uint8 0 255

    flow = calc_optical_flow_farneback(prev, next)
    # flow: (1080, 1920, 2) float64

    bgr = my_utils.flow_to_color(flow, hsv)

    if DEBUG:
        cv2.imwrite("./output_farneback.png", bgr)
        stop -= 1
        print("stop: ", stop)
    else:
        out.write(bgr)

        # Show the result on the fly
        # cv2.imshow("frame2", bgr)
        # k = cv2.waitKey(30) & 0xFF
        # if k == 27:
        #     break
        # elif k == ord("s"):
        #     cv2.imwrite("opticalfb.png", frame2)
        #     cv2.imwrite("opticalhsv.png", bgr)

    prev = next


cap.release()
out.release()
cv2.destroyAllWindows()